In [41]:
# Name: Joseph Tastet
# Date: 04/14/2023
# Purpose: Display animal shelter information based on user selections

from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter
from bson.objectid import ObjectId





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.B(html.H1('Joseph Tastet SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div(
        
        dcc.Dropdown(id='filter-type', 
                    options=[
                        {'label': 'Water Rescue', 'value': 'WR'},
                        {'label': 'Mountain Rescue', 'value': 'MR'},
                        {'label': 'Disaster Rescue', 'value': 'DR'},
                        {'label': 'Reset', 'value': 'reset'}
                    ],
                    value='WR')


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        page_size=10,
        editable=False,
        filter_action='native',
        sort_action='native',
        column_selectable=False,
        row_selectable="multi",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native"
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        #set dataframe to proper query depending on the dropdown option selected (filter_type)
        if filter_type == 'WR':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Female'},
                                                          {'$or': [
                                                            {'breed': 'Labrador Retriever Mix'},
                                                            {'breed': 'Chesa Bay Retr Mix'},
                                                            {'breed': 'Newfoundland Mix'}
                                                            ]
                                                          },
                                                          {'$and': [
                                                            {'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                            {'age_upon_outcome_in_weeks': {'$lte': 156}}
                                                            ]
                                                          }]
                                                })
                                    )
            )
                            
        elif filter_type == 'MR':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                            {'breed': 'German Shepherd'},
                                                            {'breed': 'Alaskan Malamute'},
                                                            {'breed': 'Old English Sheepdog'},
                                                            {'breed': 'Siberian Husky'},
                                                            {'breed': 'Rottweiler'}
                                                            ]
                                                          },
                                                          {'$and': [
                                                            {'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                            {'age_upon_outcome_in_weeks': {'$lte': 156}}
                                                            ]
                                                          }]
                                                })
                                    )
            )
                            
        elif filter_type == 'DR':
            df = pd.DataFrame(list(shelter.read({'$and': [{'sex_upon_outcome': 'Intact Male'},
                                                          {'$or': [
                                                            {'breed': 'Doberman Pinscher'},
                                                            {'breed': 'German Shepherd'},
                                                            {'breed': 'Golden Retriever'},
                                                            {'breed': 'Bloodhound'},
                                                            {'breed': 'Rottweiler'}
                                                            ]
                                                          },
                                                          {'$and': [
                                                            {'age_upon_outcome_in_weeks': {'$gte': 26}},
                                                            {'age_upon_outcome_in_weeks': {'$lte': 300}}
                                                            ]
                                                          }]
                                                })
                                    )
            )
                            
        elif filter_type == 'reset':
            df = pd.DataFrame.from_records(shelter.read({}))

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # imports the currently displayed data
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the breeds (breed) and values (breed.count)
    breeds = dff['breed'].value_counts().keys().tolist()
    values = dff['breed'].value_counts().tolist()
    #creates a bar graph with plotly express
    return [
        dcc.Graph(            
            figure = px.bar(
                data_frame=dff, 
                y = values, 
                x = breeds, 
                color = breeds,
                color_discrete_sequence=px.colors.qualitative.Plotly,
                title = "Number of breed",
                width=800, 
                height=500   
            )
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "selected_rows"),
    Input('datatable-id', 'selected_columns') ])
def update_map(viewData, selected_rows, selected_columns):
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    if selected_rows == []:
        return null

    #initiate future children array with the required TileLayer to start
    selected_animals = [dl.TileLayer(id="base-layer-id")]

    # for loop to make marker for each selected row then append to future child (selected_animals)
    for i in selected_rows:
        selected_animals.append(
            dl.Marker(position=[(dff.iloc[i,13]),(dff.iloc[i,14])], children=[
                    #use dff.iloc[row, column] for the breed
                    dl.Tooltip(dff.iloc[i,4]),
                    dl.Popup([
                        #use dff.iloc[row, column] for the name
                        html.H1("Animal Name"),
                        html.P(dff.iloc[i,9])
                    ])
                ])
        )


    return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=selected_animals)]


app